In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

from tqdm import tqdm
import numpy as np


python_path = "/home/yskim/projects/sparse-bfp"
import sys
if python_path not in sys.path:
    sys.path.append(python_path)

from util.reprod_util import set_reproducibility

from util.custom_linear import CustomLinear, PrecisionFlag
from util.custom_conv2d import CustomConv2d

set_reproducibility(random_seed=128)

from util.data_load import *

from util.bfp.bfp_config import BfpConfig
BfpConfig.bfp_M_Bit = 8

from util.custom_linear import LinearLogConfig
from util.custom_conv2d import Conv2dLogConfig

In [2]:
def wrap_modules(module, precision_flag: PrecisionFlag):
    for name, _ in module.named_children():
        child = getattr(module, name)
        child_cnt = 0
        for _ in child.children():
            child_cnt += 1

        if child_cnt == 0:
            layer = getattr(module, name)
            for _, param in layer.named_parameters():
                if param.requires_grad:
                    pass
                    # print(param.shape)
            # print()
            if "Linear" in str(layer):
                custom_layer = CustomLinear(input_features=layer.in_features, 
                                            output_features=layer.out_features, 
                                            bias=True if layer.bias is not None else False, 
                                            precision_flag=precision_flag)
                custom_layer.weight = nn.Parameter(layer.weight.data.clone().detach())
                if layer.bias is not None:
                    custom_layer.bias = nn.Parameter(layer.bias.data.clone().detach())
                setattr(module, name, custom_layer)
            elif "Conv" in str(layer):
                custom_layer = CustomConv2d(in_channels=layer.in_channels,
                                            out_channels=layer.out_channels,
                                            kernel_size=layer.kernel_size[0],
                                            stride=layer.stride[0],
                                            padding=layer.padding[0],
                                            bias=True if layer.bias is not None else False,
                                            precision_flag=precision_flag)
                custom_layer.weight = nn.Parameter(layer.weight.data.clone().detach())
                if layer.bias is not None:
                    custom_layer.bias = nn.Parameter(layer.bias.data.clone().detach())
                setattr(module, name, custom_layer)
        else:
            wrap_modules(child, precision_flag)

In [3]:
batch_size = 32
# train_loader, valid_loader = generate_data_loaders(type=DL_TYPE_IMAGENET, batch_size=32)
train_loader, valid_loader = generate_data_loaders(type=DL_TYPE_CIFAR_100, batch_size=batch_size)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
device = 'cpu'
BfpConfig.bfp_M_Bit = 2
CustomConv2d.current_id = 0
CustomLinear.current_id = 0

model_ref = models.resnet18(num_classes=1000)

lr = 0.01

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model_ref.parameters(), lr=lr, weight_decay=1e-2)
# optimizer = optim.SGD(model_ref.parameters(), lr=0.01)

precision_flag = PrecisionFlag.BFP
wrap_modules(model_ref, precision_flag=precision_flag)

LinearLogConfig.should_log = False
Conv2dLogConfig.should_log = False

model_ref = model_ref.to(device)
criterion = criterion.to(device)

iterations = 2
curr_iter = 0
for epoch in range(1):
    model_ref.train()
    for batch_index, (inputs, targets) in enumerate(train_loader, 0):
    # for batch_index, (inputs, targets) in enumerate(tqdm(train_loader), 0):
        if curr_iter >= iterations:
            break
    
        print(f"{curr_iter}th iter")

        train_top1_correct_cnt = 0
        train_top5_correct_cnt = 0
        train_total_cnt = 0
        train_loss = 0

        inputs = inputs.to(device)
        targets = targets.to(device)

        outputs = model_ref(inputs)

        pred_labels = torch.argmax(outputs, axis=1)
        
        train_top1_correct_cnt += torch.sum(pred_labels == targets).item()

        _, indices = torch.topk(outputs, k=5)
        for b in range(outputs.shape[0]):
            if targets[b] in indices[b]:
                train_top5_correct_cnt += 1
                continue

        train_total_cnt += outputs.shape[0]

        loss = criterion(outputs, targets)

        train_loss += loss.item() * outputs.shape[0]

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss /= train_total_cnt
        train_top1_acc = train_top1_correct_cnt / train_total_cnt * 100.0
        train_top5_acc = train_top5_correct_cnt / train_total_cnt * 100.0

        print(f"[{curr_iter}th iter train accuracy] "
              f"top1: {train_top1_acc:.2f}%, "
              f"top5: {train_top5_acc:.2f}%, "
              f"loss: {train_loss}")
        print()
            

        curr_iter += 1

0th iter
fwd im2col time: 83.61780ms
fwd weight reshaped time: 0.01154ms
fwd gemm time: 155.16832ms
fwd output permute time: 21.397652ms
fwd im2col time: 27.83273ms
fwd weight reshaped time: 0.00916ms
fwd gemm time: 5.13866ms
fwd output permute time: 13.141629ms
fwd im2col time: 25.62250ms
fwd weight reshaped time: 0.00770ms
fwd gemm time: 5.17251ms
fwd output permute time: 18.509062ms
fwd im2col time: 25.37399ms
fwd weight reshaped time: 0.00722ms
fwd gemm time: 5.15386ms
fwd output permute time: 19.004827ms
fwd im2col time: 30.44309ms
fwd weight reshaped time: 0.00742ms
fwd gemm time: 5.21058ms
fwd output permute time: 19.682737ms
fwd im2col time: 9.91463ms
fwd weight reshaped time: 0.11384ms
fwd gemm time: 2.31450ms
fwd output permute time: 4.478264ms
fwd im2col time: 8.11694ms
fwd weight reshaped time: 4.15304ms
fwd gemm time: 4.35419ms
fwd output permute time: 15.338933ms
fwd im2col time: 3.10367ms
fwd weight reshaped time: 5.70260ms
fwd gemm time: 0.46851ms
fwd output permute tim